In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

%load_ext autoreload
%autoreload 2

# Trajectory analysis with RANSAC on heatmaps

In [ ]:
from train_configurations import tracknet_v2
from trajectories.data_reading import get_frame, get_heatmap, get_candidates

In [ ]:
starting_frame, candidates, n_candidates, values = get_candidates(tracknet_v2)

In [ ]:
i = 0
frame = get_frame(tracknet_v2, starting_frame + i)
heatmap = get_heatmap(tracknet_v2, starting_frame + i)

plt.imshow(frame)
plt.imshow(cv2.resize(heatmap, (1280, 720)), alpha=0.7, cmap='gray', vmin=0, vmax=1)

## Fit parabolic trajectories

In [ ]:
from trajectories.fitting import *
from trajectories.filtering import *
from trajectories.visualization import *
from trajectories.utils import Graph

In [ ]:
trajectory_info = fit_trajectories(candidates, n_candidates, starting_frame)
trajectories_to_json(trajectory_info, 'bbb.json', indent=1)

## Build trajectory graph

In [ ]:
from trajectories.filtering import build_trajectory_graph
trajectory_graph = build_trajectory_graph(trajectory_info)

### Visualize Trajectory graph

In [ ]:
from trajectories.visualization import visualize_trajectory_graph
ax = visualize_trajectory_graph(trajectory_graph, 814)
plt.show()

In [ ]:
from trajectories.filtering import find_shortest_paths
shortest_paths = find_shortest_paths(trajectory_graph)

## Visualize on frame

In [ ]:
shortest_paths.__len__()

In [ ]:
ti = np.array([t['k_seed'] for t in trajectory_info])

dpi = 50
fig, ax = plt.subplots(figsize=(w_frame/dpi, h_frame/dpi), dpi=dpi)

# colors = ['r', 'g', 'y', 'k']
colors = ['y', 'w']

ax.imshow(get_frame(video_source, detection_df_part['frame_num'][detection_df_part.index[2402]]), zorder=-2)

to_print = []
for i, node in enumerate(shortest_paths[0]):
    if node not in range(2399, 2419):
        continue

    to_print.append(node)
    df_index = np.where(ti==node)[0][0]
    trajectory = trajectory_info[df_index]['trajectory']

    k_min = trajectory_info[df_index]['k_min']
    try:
        next_index = np.where(ti==shortest_paths[0][i+1])[0][0]
        k_max = trajectory_info[next_index]['k_min']
    except IndexError as e:
        k_max = trajectory_info[df_index]['k_max']

    show_fit(trajectory*frt, candidates*frt,
             k_min=k_min,
             k_mid=trajectory_info[df_index]['k_mid'],
             k_max=k_max,
             i_min=trajectory_info[df_index]['i_min'],
             i_mid=trajectory_info[df_index]['i_mid'],
             i_max=trajectory_info[df_index]['i_max'],
             k_seed=trajectory_info[df_index]['k_seed'],
             i_seed=trajectory_info[df_index]['i_seed'],
             ax=ax,
             annotate=True,
             show_fitting_points=True,
             trajectory_color=colors[i%len(colors)])# 'y' if i%2==0 else 'r')
ax.set_xlim(0, w_frame)
ax.set_ylim(h_frame, 0)

print(to_print)
plt.show()

In [ ]:
ti = np.array([t['k_seed'] for t in trajectory_info])

dpi = 50
fig, ax = plt.subplots(figsize=(w_frame/dpi, h_frame/dpi), dpi=dpi)

colors = ['r', 'w', 'g', 'k']
ax.imshow(get_frame(video_source, detection_df_part['frame_num'][detection_df_part.index[99]]), zorder=-2)

# for i, frame in enumerate((107, 115, 123)):
for i, frame in enumerate((100, 101)):
# for i, frame in enumerate((77,)):
    df_index = np.where(ti==frame)[0][0]
    trajectory = trajectory_info[df_index]['trajectory']


    show_fit(trajectory*frt, candidates*frt, *[trajectory_info[df_index][key] for key in info_keys[:-2]],
             ax=ax,
             annotate=True,
             show_fitting_points=True,
             trajectory_color=colors[i%len(colors)])# 'y' if i%2==0 else 'r')
    ax.set_xlim(0, w_frame)
    ax.set_ylim(h_frame, 0)

plt.show()

# Visualize some activations and kernels because why not

In [ ]:
model = TrackNetV2RNN(sequence_length=4)
model.load('checkpoints/tracknet_v2_rnn_360_640/phase_3_0/checkpoint_0002_best.ckpt')
model.eval()
model

In [ ]:
dataset_params = dict(image_size=(360, 640),
                      sequence_length=4,
                      sigma=5,
                      drop_duplicate_frames=False,
                      transform = ToTensor(),
                      target_transform = ToTensor(),
                      grayscale=False)

dataset = VideoDatasetRNN(root="../datasets/prova/", **dataset_params)

In [ ]:
counter = 0

def get_encoding_layer(desired_block=1, subblock=0):
    layers = []
    for i, block in enumerate(model.children()):
        # print(i)
        if i%2 == 1:
            layers.append(block)
        for j, block_element in enumerate(block.children()):
            #print(i, j)
            for k, layer in enumerate(block_element.children()):
                layers.append(layer)
                # print(i, j, k)
                if type(layer) is torch.nn.ReLU and i==2*desired_block and j==subblock:
                    break
            if type(layer) is torch.nn.ReLU and i==2*desired_block and j==subblock:
                break
        if type(layer) is torch.nn.ReLU and i==2*desired_block:
            break
    return layers

def compute_activations(layers, input):
    activation = input.unsqueeze(dim=0)
    with torch.no_grad():
        for l in layers:
            activation = l(activation)

    return activation.squeeze().numpy()

In [ ]:
frames, labels = dataset[50]
frames = frames.to(torch.float32)

In [ ]:
w, h_frame, dpi = 300*2*16/9, 300, 100

fig, axs = plt.subplots(ncols=2, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

axs[0].imshow(frames[-3:].numpy().transpose(1, 2, 0))
axs[0].set_title("Input frame (last in sequence)")

axs[1].imshow(labels[0])
axs[1].set_title("Ground truth")

fig.tight_layout(pad=0.2)
plt.show()

In [ ]:
noise_part = np.linspace(0, 1, 10)
c = []

for n in noise_part:
    with torch.no_grad():
        f = (1-n)*frames + n*torch.randn(frames.shape)
        out = model(f.unsqueeze(dim=0)).squeeze().numpy()
    c.append(out.max())
plt.plot(noise_part, c)

In [ ]:
n = 0.07
with torch.no_grad():
    f = (1-n)*frames + n*torch.randn(frames.shape)
    out = model(f.unsqueeze(dim=0)).squeeze().numpy()
plt.imshow(out)
plt.colorbar()
plt.show()

In [ ]:
frames[:3] = torch.zeros(3, 360, 640)

In [ ]:
block = 2
subblock = 1

activations = compute_activations(get_encoding_layer(block, subblock), frames)
activations.shape

In [ ]:
(dead_activations, ) = np.where(activations.max(axis=(1,2))==0)
print(f"Of {activations.shape[0]} activations, {dead_activations.size} are dead and {activations.shape[0]-dead_activations.size} are not.")

In [ ]:
height_pixels = 1080
top_adjust = 1

w, h_frame, dpi = height_pixels*16/9*top_adjust, height_pixels, 100
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

i_0 = 0

for k, ax in enumerate(axs.ravel()):
    ax.imshow(activations[k+i_0], cmap='gray')
    # ax.set_title(i)
    ax.set_axis_off()

#fig.suptitle(f"Activations in encoding block {block}, subblock {subblock}")

fig.tight_layout(pad=0.5)
fig.subplots_adjust(top=top_adjust)

#fig.savefig(f"{block}_{subblock}.png")

plt.show()

In [ ]:
model.state_dict().keys()

In [ ]:
dk = 4

kernels = model.state_dict()['vgg_conv1.1.0.weight'].numpy()
biases = model.state_dict()['vgg_conv1.1.0.bias'].numpy()
w, h_frame, dpi = 800, 800, 100
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

print(kernels.shape)
print(biases[dk])

min_val = kernels[dk].min()
max_val = kernels[dk].max()
print(min_val, max_val)

max_val=max((max_val, -min_val))
min_val=min((-max_val, min_val))

for k, ax in enumerate(axs.ravel()):
    ax.imshow(kernels[dk,k], cmap='RdBu', vmin=min_val, vmax=max_val)
    ax.set_axis_off()

#fig.suptitle(f"Kernel {k}, bias = {biases[k]:.2g}")
fig.tight_layout(pad=0.2)
plt.show()
